# Showcase of Bybit Future - Future calc
The Bybitmaster has custom methods that I coded, but can also make normal request to the Bybit API with the session attribute.

In [3]:
# Inside of the module, we use absolute imports
# Because this context is different, we need to add inside the PYTHONPATH the path of the module
import sys
sys.path.append('Bybit')

from Bybit.greekMaster import SpotFutStrategos
import Bybit.utils
import Bybit.simulator
Bybit.utils.ColorFormatter.configure_logging(run_name="save_klines.log", verbose=1)

# Create a Bybit master
master = SpotFutStrategos(demo=True)
sim = Bybit.simulator.Simulator()
master.fetcher.get_USDC_BTC()

2025-01-18 09:43:53,688 - root - INFO - Global logging configuration applied with verbosity level 1
2025-01-18 09:43:53,689 - greekMaster - INFO - GreekMaster initialized


InvalidRequestError: API key is invalid. (ErrCode: 10003) (ErrTime: 09:43:54).
Request → GET https://api-demo.bybit.com/v5/account/wallet-balance: accountType=UNIFIED&coin=BTC.

In [2]:
# Get all futures BTC gaps
DataWrangler = master.fetcher.all_gaps_pd(inverse=True, spot=True, pretty=True, applyFees=True)
DataWrangler

,Buy,Sell,Gap,Coeff,APR,DaysLeft,CumVolume
0,BTCUSDC (Spot),BTCUSDT (Spot),$ 8.13,0.00 %,-0.00 %,-20105,3.63B
1,BTCUSDC (Spot),BTCUSDH25,$ 2661.81,2.36 %,12.32 %,69,132.26M
2,BTCUSDC (Spot),BTCUSDM25,$ 6012.81,5.61 %,12.73 %,160,132.26M
3,BTCUSDC (Spot),BTC-24JAN25,$ 364.81,0.13 %,7.06 %,6,136.91M
4,BTCUSDC (Spot),BTC-31JAN25,$ 584.31,0.35 %,9.09 %,13,133.38M
...,...,...,...,...,...,...,...
61,BTC-28MAR25,BTC-26SEP25,$ 6507.00,5.93 %,30.95 %,69,4.54M
62,BTC-28MAR25,BTC-26DEC25,$ 9502.00,8.76 %,45.73 %,69,3.45M
63,BTC-27JUN25,BTC-26SEP25,$ 3132.00,2.65 %,6.01 %,160,3.89M
64,BTC-27JUN25,BTC-26DEC25,$ 6127.00,5.39 %,12.23 %,160,2.80M


In [ ]:
await master.client.fetcher.get_history_pd("BTC-17JAN25", category="linear", interval="15", dest="store")
await master.client.fetcher.get_history_pd("BTC-17JAN25", category="linear", interval="5", dest="store")
await master.client.fetcher.get_history_pd("BTC-17JAN25", category="linear", interval="1", dest="store")

# master.client.fetcher.session.get_instruments_info(symbol="BTCUSDH25", category="inverse")

# Bybit.utils.get_date(1743148800000)
# start = Bybit.utils.get_date(1726214400000)

## Tester
Just to run some tests

In [ ]:
# 1 - Simple enter position logic (given a future and a perpetual)
# 2 - Simple exit position logic (given a future and a perpetual). Can also exit when reaching a satisfying unexpected gap
# 3 - Logic to activate both operations (given a future and a perpetual). Enter again when reaching a satisfying unexpected gap
# 4 - Logic to find the best future and perpetual to trade
# 5 - Logic to keep the loop running
# 6 - Monitoring features
# 7 - Use systemctl to launch 24/7
# We use sockets, once we selected the best local future, compared to their average gap.
# Shared state to store the latest updates
import datetime
print(datetime.datetime.now().microsecond)
(1734816631000 - datetime.datetime.now().microsecond) / 1000

## Fetching time :)

Getting 15, 5 and 1 minute candles for everything since 01/01/2024

In [ ]:
# Future contracts
await master.client.fetcher.save_klines(dest="store")

2025-01-17 11:39:13,877 - greekMaster.client.fetcher - INFO - Fetching data for BTC-24JAN25.
2025-01-17 11:39:14,118 - greekMaster.client.fetcher - INFO - Loaded 1294 existing data points.
2025-01-17 11:39:14,317 - greekMaster.client.fetcher - INFO - Fetched 66 new data points.
2025-01-17 11:39:14,330 - greekMaster.client.fetcher - INFO - Fetching data for BTC-24JAN25.
2025-01-17 11:39:14,341 - greekMaster.client.fetcher - INFO - Loaded 3880 existing data points.
2025-01-17 11:39:14,705 - greekMaster.client.fetcher - INFO - Fetched 197 new data points.
2025-01-17 11:39:14,715 - greekMaster.client.fetcher - INFO - Fetching data for BTC-24JAN25.
2025-01-17 11:39:14,730 - greekMaster.client.fetcher - INFO - Loaded 19400 existing data points.
2025-01-17 11:39:15,268 - greekMaster.client.fetcher - INFO - Fetched 981 new data points.
2025-01-17 11:39:15,316 - greekMaster.client.fetcher - INFO - Fetching data for BTC-31JAN25.
2025-01-17 11:39:15,337 - greekMaster.client.fetcher - INFO - Loade

## Plot testing

Part to test the plot of the data.


In [ ]:
# Yes
# master.client.fetcher.get_futureNames(quoteCoins=["USDT"])

## Long Spot x Short Future

Spot USDC is 0% fees and Future is 0.0550% taker fees (0% with contract delivery).
Problem is that spot cannot be used with leverage. Could borrow but that would be awkward. (tried it)

In [ ]:
Bybit.utils.load_klines_parquet("store/BTC-20DEC24_1.parquet", pretty=True)

sim.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-20DEC24_1.parquet")

In [ ]:
# Long position, then short position
sim.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-15NOV24_1.parquet")

## Long Perpetual x Short Future

Perpetual USDT is 0.0550% taker fees and Future is 0.0550% taker fees (0% with contract delivery).
Yes we have fees, but wen can use leverage on both sides.

We use Perpetual USDT because there is more liquidity.

In [ ]:
sim.plot_compare("store/BTCPERP_5.parquet", "store/BTC-29NOV24_5.parquet")

In [ ]:
sim.plot_compare("store/BTCUSDT_5.parquet", "store/BTC-29NOV24_5.parquet")

## Long Spot x Short Inverse Future

What if I buy spot and collateralize it to short inverse future?
We effectively capture the difference.

In [ ]:
# Keep going when get_history_pd file names are changed

## OKX 20x Leverage test

I launched a test with OKX with a short on BTCUSDT-27JUN25 and long on BTCUSDT-29NOV24.

Following section will track the the two contracts' data

In [ ]:
sim.plot_compare("store/BTC-29NOV24_15.parquet", "store/BTC-27JUN25_15.parquet", dateLimit="2024-05-06")